In [1]:
import numpy as np
import cv2
from mss import mss
from PIL import Image

import pyautogui
import time

import torchvision.models.segmentation
import torch
import torchvision.transforms as tf
import torch.nn as nn
import torch.nn.functional as F

In [2]:
modelsPath = 'Models'
modelName = 'bestModel.torch'
pathToModel = f'{modelsPath}/{modelName}'

#wheel crop area
x1W = 812
x2W = 932
y1W = 913
y2W = 1033

#wheel image size
widthWheel  = y2W - y1W
heightWheel = x2W-x1W


bounding_box = {'top': 340, 'left': 1490, 'width': 1920, 'height': 1080}

In [3]:
class net(nn.Module):
    def __init__(self):
        super(net, self).__init__()
        self.fc = nn.Linear(512, 128)
        
        self.branch_a1 = nn.Linear(128, 32)
        self.branch_a2 = nn.Linear(32, 1)
    
    def forward(self, x):
        x = F.leaky_relu(self.fc(x))

        a = F.leaky_relu(self.branch_a1(x))
        
        out1 = self.branch_a2(a)
        
        return out1

#load net
#To Load Pretrained Weights:   weights='ResNet18_Weights.DEFAULT'
resnet18 = torchvision.models.resnet18()
resnet18.fc = nn.Identity()
net_add=net()
model = nn.Sequential(resnet18, net_add)

# Set device GPU or CPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu') 

# load model to GPU
model = model.to(device)

model.load_state_dict(torch.load(pathToModel))

model.eval()

Sequential(
  (0): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runnin

In [4]:
def directionToSteer(current,target):
    difference = current-target
    if difference > 0:
        pyautogui.typewrite('a')
    elif difference < 0:
        pyautogui.typewrite('d')

In [5]:
sct = mss()

time.sleep(10)

pyautogui.typewrite('1')

while True:
    sct_img = sct.grab(bounding_box)
    wholeFrame = np.array(sct_img)

    #crop to steering wheel logo and transform image to black and white but in RGB color space
    crop = wholeFrame[x1W:x2W, y1W:y2W]

    grayImage = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)
    (thresh, contrastImg) = cv2.threshold(grayImage, 100, 255, cv2.THRESH_BINARY)
    backtorgb = cv2.cvtColor(contrastImg,cv2.COLOR_GRAY2RGB)

    transformImg=tf.Compose([tf.ToPILImage(),tf.Resize((heightWheel,widthWheel)),tf.ToTensor(),tf.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))]) 
    wheel=transformImg(np.array(backtorgb))

    wheel = wheel.to(device).unsqueeze(0)

    with torch.no_grad():
        prediction = model(wheel)

    detectedAngle = prediction.data.cpu().numpy()[0][0]

    realAngle = (180*detectedAngle)-90

    #directionToSteer(realAngle,180)

    print("Steering Angle: ", realAngle)

    cv2.imshow('screen', np.array(crop))
    
    if (cv2.waitKey(1) & 0xFF) == ord('q'):
        cv2.destroyAllWindows()
        break

Steering Angle:  -4.3611860275268555
Steering Angle:  -11.360195875167847
Steering Angle:  -13.142555952072144
Steering Angle:  -13.142555952072144
Steering Angle:  -12.329460382461548
Steering Angle:  -11.13216519355774
Steering Angle:  -11.13216519355774
Steering Angle:  -10.032212734222412
Steering Angle:  -6.436035633087158
Steering Angle:  -5.685789585113525
Steering Angle:  -9.537656307220459
Steering Angle:  -6.355569362640381
Steering Angle:  -11.00657343864441
Steering Angle:  -8.643407821655273
Steering Angle:  -11.946339011192322
Steering Angle:  -12.2133207321167
Steering Angle:  -12.2133207321167
Steering Angle:  -4.088453650474548
Steering Angle:  -5.377609133720398
Steering Angle:  1.8633198738098145
Steering Angle:  -2.652468681335449
Steering Angle:  -6.304639577865601
Steering Angle:  -3.550257682800293
Steering Angle:  -4.60552453994751
Steering Angle:  -4.238555431365967
Steering Angle:  -4.273349046707153
Steering Angle:  -4.273349046707153
Steering Angle:  -4.2733

KeyboardInterrupt: 